<a href="https://colab.research.google.com/github/weslei-silva87/Analise-Pevisao-vendas-farmaceutica/blob/main/Farmacia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



In [ ]:
tabela = pd.read_csv('/content/sales mensal.csv')
display(tabela)

Plotando Grafico

In [ ]:
# lista com todas as médias e seus respectivos nomes
medias = [media_M01AB, media_M01AE, media_N02BA, media_N02BE, media_N05B, media_N05C, media_R03, media_R06]
nomes_medicamentos = ['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06']

# figura e eixos para os subplots
fig, axs = plt.subplots(2, 4, figsize=(20, 10))

# Plote os gráficos para cada medicamento
for i, media in enumerate(medias):
    linha = i // 4
    coluna = i % 4
    axs[linha, coluna].plot(media.index, media.values, marker='o', linestyle='-')
    axs[linha, coluna].set_title(f'Média de demanda para o medicamento {nomes_medicamentos[i]} por ano')
    axs[linha, coluna].set_xlabel('Ano')
    axs[linha, coluna].set_ylabel('Média de demanda')

# Ajuste o layout para evitar sobreposição de texto
plt.tight_layout()

# Exiba o gráfico
plt.show()



Medicamentos Crescimento e Queda


In [ ]:
# Inicializando a lista de resultados
resultados = []

# Verificando se cada medicamento aumentou ou diminuiu sua demanda
for medicamento, demanda_final in media_demandas.items():
    demanda_inicial = tabela_nova.iloc[0][medicamento]  # Demanda no primeiro ano

    if demanda_final > demanda_inicial:
        aumento_percentual = ((demanda_final - demanda_inicial) / demanda_inicial) * 100
        resultados.append((medicamento, aumento_percentual))
    elif demanda_final < demanda_inicial:
        diminuicao_percentual = ((demanda_inicial - demanda_final) / demanda_inicial) * 100
        resultados.append((medicamento, -diminuicao_percentual))
    else:
        resultados.append((medicamento, 0))

# Plotando o gráfico de barras
fig, ax = plt.subplots(figsize=(10, 6))
barras = ax.bar([x[0] for x in resultados], [x[1] for x in resultados], color=['red' if x[1] < 0 else 'blue' for x in resultados])

# Adicionando os valores nas barras
for i, barra in enumerate(barras):
    altura = barra.get_height()
    ax.annotate('{:.2f}%'.format(altura),
                xy=(barra.get_x() + barra.get_width() / 2, altura),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

# Definindo rótulos e título
ax.set_ylabel('Variação Percentual')
ax.set_title('Variação Percentual da Demanda de Medicamentos')
plt.xticks(rotation=45, ha='right')

plt.show()


Analise de sazonalidade


In [ ]:

# Agrupando os dados por ano e mês e somando as vendas
vendas_ano = tabela_nova.groupby(['ano', 'mes']).sum()

# Criando um DataFrame para facilitar a manipulação dos dados
df_vendas = vendas_ano.reset_index()

# Criando o gráfico interativo
fig = go.Figure()

# Adicionando uma linha para cada ano
for ano in df_vendas['ano'].unique():
    df_ano = df_vendas[df_vendas['ano'] == ano]
    fig.add_trace(go.Scatter(x=df_ano['mes'], y=df_ano['Faturamento'], mode='lines+markers', name=str(ano)))

# Adicionando layout ao gráfico
fig.update_layout(title='Vendas ao longo do tempo',
                  xaxis_title='Mês',
                  yaxis_title='Faturamento',
                  legend_title='Ano')

# Exibindo o gráfico
fig.show()


 Quais as classes de medicamentos mais e menos vendidos por mês?¶

In [ ]:

# Definindo as colunas de medicamentos
medicamentos = ['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06']

# Calculando a soma de vendas de cada medicamento por mês
vendas_por_mes = tabela_nova.groupby(['mes', 'ano'])[medicamentos].sum()

# Calculando os medicamentos mais e menos vendidos por mês
mais_vendidos_por_mes = vendas_por_mes.idxmax(axis=1)
menos_vendidos_por_mes = vendas_por_mes.idxmin(axis=1)

# Calculando a porcentagem de vendas de cada medicamento
total_vendas = vendas_por_mes.sum(axis=1)
percentual_mais_vendidos = (vendas_por_mes.max(axis=1) / total_vendas) * 100
percentual_menos_vendidos = (vendas_por_mes.min(axis=1) / total_vendas) * 100

# Criando um DataFrame para exibir os resultados como tabela
resultados = pd.DataFrame({
    'Medicamentos mais vendidos': mais_vendidos_por_mes,
    'Porcentagem mais vendidos (%)': percentual_mais_vendidos,
    'Medicamentos menos vendidos': menos_vendidos_por_mes,
    'Porcentagem menos vendidos (%)': percentual_menos_vendidos
})

# Exibindo os resultados como tabela
print("Resultados:")
print(resultados)

In [ ]:
Modelo Preditivo

In [ ]:

# Definir as colunas de recursos
colunas_classes = ['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06']

# Definir a coluna de alvo (vendas totais ou faturamento)
alvo = 'Faturamento'

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(tabela_nova[colunas_classes], tabela_nova[alvo], test_size=0.2, random_state=42)

# Inicializar os modelos de regressão
modelo_linear = LinearRegression()
modelo_arvore = DecisionTreeRegressor()

# Treinar os modelos
modelo_linear.fit(X_train, y_train)
modelo_arvore.fit(X_train, y_train)

# Fazer previsões nos conjuntos de teste
predicoes_linear = modelo_linear.predict(X_test)
predicoes_arvore = modelo_arvore.predict(X_test)

# Avaliar os modelos usando métricas de erro
mae_linear = mean_absolute_error(y_test, predicoes_linear)
mse_linear = mean_squared_error(y_test, predicoes_linear)
mae_arvore = mean_absolute_error(y_test, predicoes_arvore)
mse_arvore = mean_squared_error(y_test, predicoes_arvore)

# Imprimir os resultados
print("Resultados do Modelo de Regressão Linear:")
print("MAE:", mae_linear)
print("MSE:", mse_linear)
print("\nResultados do Modelo de Árvore de Decisão:")
print("MAE:", mae_arvore)
print("MSE:", mse_arvore)

Previsões de faturamento

In [ ]:
# Criar o gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(classes, faturamento_previsto, color='skyblue')

# Adicionar os valores acima das barras
for i, valor in enumerate(faturamento_previsto):
    plt.text(i, valor + 0.1, str(valor), ha='center', va='bottom')

# Configurações do gráfico
plt.xlabel('Classes de Medicamentos')
plt.ylabel('Faturamento Previsto')
plt.title('Faturamento Previsto por Classe de Medicamento')
plt.xticks(rotation=45)
plt.grid(axis='y')

# Mostrar o gráfico
plt.tight_layout()
plt.show()

# Imprimir o faturamento previsto
print("Faturamento Previsto:", faturamento_previsto)